In [1]:
# inspired by https://github.com/pommi/python-itho-wpu/blob/master/convert-itho-db.py

PARFILE = ".\parameters_HeatPump.mdb"
PASSWORD = 'itho_parameters'

import pyodbc, re
from collections import defaultdict
import slugify

In [2]:
# zoek de driver
# https://github.com/mkleehammer/pyodbc/wiki/Connecting-to-Microsoft-Access
[x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]

['Microsoft Access Driver (*.mdb, *.accdb)']

## Lees MDB

In [63]:
MDB = PARFILE
DRV = '{Microsoft Access Driver (*.mdb, *.accdb)}'


# connect to db
conn = pyodbc.connect(f'DRIVER={DRV};DBQ={PARFILE};PWD={PASSWORD}')
conn.setencoding("UTF-8")
conn.setdecoding(pyodbc.SQL_CHAR, encoding="UTF-8")
cur = conn.cursor()


In [64]:
tables = []
for table_info in cur.tables(tableType="TABLE"):
        if re.match(
            "^(VersieBeheer|Data[Ll]abel|Parameterlijst|Handbed|Counters)", table_info.table_name
        ):
            tables.append(table_info.table_name)
tables[:5]

['Counters_V10',
 'Counters_V2',
 'Counters_V37',
 'DataLabel_V10',
 'DataLabel_V11']

In [65]:
def get_version(s):
    x = re.findall('\d+', s)
    assert len(x) == 1
    return int(x[0])

assert get_version("Parameterlijst_V10")==10

In [92]:
WPUSettingsLabels = defaultdict(list)
WPUStatusLabels = defaultdict(list)
Versies = []

StatusLabelNames = []  # "Naam" column in DataLabel_Vxx tables StatusLabelName['T_out'] = 0
StatusLabels = {}  # StatusLabels['T_out'] = ('Outside Temp (C)', 'outside-temp-c') 


for table in sorted(tables):
    #print(table)
    
    if re.match("^Parameterlijst", table):
        fw_ver = get_version(table)
        cur.execute(f"select Index, Naam, Tekst_NL, Tekst_GB, Eenheid_NL, Eenheid_GB from {table}") 
        rows = cur.fetchall()
        for r in sorted(rows):
            WPUSettingsLabels[fw_ver].append((r.Index, r.Naam, r.Tekst_NL, r.Eenheid_NL, r.Tekst_GB, r.Eenheid_GB))
    if re.match("^Data[Ll]abel", table):
        fw_ver = get_version(table)
        cur.execute(f"select Index, Naam, Tekst_NL, Tekst_GB, Eenheid_NL, Eenheid_GB from {table}")
        rows = cur.fetchall()
        for r in sorted(rows):
            WPUStatusLabels[fw_ver].append((r.Index, r.Naam, r.Tekst_NL, r.Eenheid_NL, r.Tekst_GB, r.Eenheid_GB))          
    if re.match("^VersieBeheer", table):
        cur.execute(f"select VersieNummer, DataLabel, ParameterLijst, Handbed, Counters from {table}")
        rows = cur.fetchall()
        for r in sorted(rows):
            Versies.append((r.VersieNummer, r.DataLabel, r.ParameterLijst, r.Handbed, r.Counters))        
#sorted(WPUSettingsLabels.keys()), sorted(WPUStatusLabels.keys())    

In [93]:
# get all versions of tables for each firmware

DataLabels = {}
SettingLabels = {}

for versie in Versies:
    fw, datalabel, setting, handbed, counter = versie
    DataLabels[datalabel] = datalabel
    SettingLabels[setting] = setting
fw_versions_dl= sorted([int(x) for x in DataLabels.keys()])
fw_versions_settings = sorted([int(x) for x in SettingLabels.keys()])

print(fw_versions_settings)

[2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 31, 33, 34, 37, 41]


## Data Labels

In [94]:
WPUStatusLabels[2][:5]

[(0, 'T_out', 'Buitentemp', '°C', None, None),
 (1, 'T_BoilDwn', 'Boiler laag', '°C', None, None),
 (2, 'T_BoilTop', 'Boiler hoog', '°C', None, None),
 (3, 'T_Evap', 'Verdamper temp', '°C', None, None),
 (4, 'T_Suct', 'Zuiggas temp', '°C', None, None)]

In [95]:
def itho_WPU_status_line(fw_ver, l):
    s = 'const uint8_t itho_WPUstatus'
    s += str(fw_ver)
    s += '[]{'
    s += ', '.join(str(x) for x in l)
    s += ', 255};'
    return s

print(itho_WPU_status_line(42, [3, 99, 42]))

const uint8_t itho_WPUstatus42[]{3, 99, 42, 255};


In [96]:
print("// diff this against wpu.h. Only 8_9 11_17 and 18_19 are different. All keys should be the same")
StatusLabelNames = []
for fw_ver in fw_versions_dl:
    fw_ver = int(fw_ver)
    #print(fw_ver)
    label_list = []
    for label in WPUStatusLabels[fw_ver]:
        idx, name, _,_,_,_ = label
        #print(idx, name)
        if name not in StatusLabelNames:
            StatusLabelNames.append(name)
        list_idx = StatusLabelNames.index(name)
        label_list.append(list_idx)
    print(itho_WPU_status_line(fw_ver, label_list))

// diff this against wpu.h. Only 8_9 11_17 and 18_19 are different. All keys should be the same
const uint8_t itho_WPUstatus2[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 255};
const uint8_t itho_WPUstatus4[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 255};
const uint8_t itho_WPUstatus5[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 68, 255};
const uint8_t itho_WPUstatus6[]{0, 1, 2, 3, 4, 5, 6, 7, 

In [97]:
Labels = {}

# create lookup tabel for Statuslabels, based on most recent firmware
for fw in fw_versions_dl:
  for label in WPUStatusLabels[fw]:
    idx, name, nl_tekst, nl_unit, gb_tekst, gb_unit = label
    #print(name, gb_tekst, gb_unit)
    if gb_tekst is None:
        gb_tekst = nl_tekst
        gb_unit = nl_unit
    if gb_unit is None:
        unit = ""
    else:
        unit = " ("+gb_unit+")"
    desc = gb_tekst+unit
    slug = str(slugify.slugify(desc))
    Labels[name] = (desc,slug)

Labels['T_out']

('Outside temp (°C)', 'outside-temp-degc')

In [98]:
print("const struct ithoLabels ithoWPUStatusLabels[]{")
for idx,label in enumerate(StatusLabelNames):
    desc, slug = Labels[label]
    print("    {\""+desc+", "+slug+"},")

const struct ithoLabels ithoWPUStatusLabels[]{
    {"Outside temp (°C), outside-temp-degc},
    {"boilertemp under (°C), boilertemp-under-degc},
    {"boilertemp above (°C), boilertemp-above-degc},
    {"evaporator temp (°C), evaporator-temp-degc},
    {"suction gas temp (°C), suction-gas-temp-degc},
    {"compressed gas (°C), compressed-gas-degc},
    {"liquid temp (°C), liquid-temp-degc},
    {"temp to source (°C), temp-to-source-degc},
    {"temp form source (°C), temp-form-source-degc},
    {"CH supply temp (°C), ch-supply-temp-degc},
    {"CH return temp (°C), ch-return-temp-degc},
    {"CH pressure (Bar), ch-pressure-bar},
    {"current compressor (A), current-compressor-a},
    {"current e-element (A), current-e-element-a},
    {"Pressure switch, pressure-switch},
    {"Tariff, tariff},
    {"condensation protection, condensation-protection},
    {"pulsteller, pulsteller},
    {"Flow sensor (lt/hr), flow-sensor-lt-hr},
    {"phase detection, phase-detection},
    {"CH pump (%), 

## SettingLabels

In [99]:
print(fw_versions_settings)

[2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 31, 33, 34, 37, 41]


In [100]:
def itho_WPU_setting_line(fw_ver, l):
    s = 'const uint16_t itho_WPUsetting'
    s += str(fw_ver)
    s += '[]{'
    s += ', '.join(str(x) for x in l)
    s += ', 999};'
    return s

print(itho_WPU_setting_line(42, [3, 99, 42]))

const uint16_t itho_WPUsetting42[]{3, 99, 42, 999};


In [106]:
print("// diff this against wpu.h.")
SettingLabelNames = []
for fw_ver in fw_versions_settings:
    fw_ver = int(fw_ver)
    #print(fw_ver)
    label_list = []
    for label in WPUSettingsLabels[fw_ver]:
        idx, name, _,_,_,_ = label
        #print(idx, name)
        if name not in SettingLabelNames:
            SettingLabelNames.append(name)
        list_idx = SettingLabelNames.index(name)
        label_list.append(list_idx)
    print(itho_WPU_setting_line(fw_ver, label_list))

// diff this against wpu.h.
const uint16_t itho_WPUsetting2[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 999};
const uint16_t itho_WPUsetting4[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 108, 109, 110, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 111, 56, 57, 112, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 113, 114, 115, 116, 117, 118, 119, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 

In [113]:
Labels = {}

# create lookup tabel for Statuslabels, based on most recent firmware
for fw in fw_versions_settings:
  for label in WPUSettingsLabels[fw]:
    idx, name, nl_tekst, nl_unit, gb_tekst, gb_unit = label
    #print(name, gb_tekst, gb_unit)
    if gb_tekst is None:
        gb_tekst = nl_tekst
        gb_unit = nl_unit
    if gb_unit is None:
        unit = ""
    else:
        unit = " ("+gb_unit+")"
    desc = gb_tekst+unit
    Labels[name] = desc

Labels['Spare']

'Spare'

In [117]:
print("const char *ithoWPUSettingsLabels[] = {")
for idx,label in enumerate(SettingLabelNames):
    desc = Labels[label]
    print(idx+34,"    \""+desc+",")

const char *ithoWPUSettingsLabels[] = {
34     "niet gebruikt,
35     "Hardware configuration,
36     "Year of commissioning,
37     "Date of commissioning,
38     "max manaul opeation time (min),
39     "frost temp (°C),
40     "offset for forst temp (K),
41     "differential vorst temp electrical element (K),
42     "Error reset time (min),
43     "Log interval (sec),
44     "switch-on delay (sec),
45     "Speed expansion valve (ticks),
46     "Max open expansion valve (puls),
47     "Extra closing pulses (puls),
48     "waiting time first valve adjustment (sec),
49     "Expansion valve corrention interval (sec),
50     "Min pluse expansion valve at start (puls),
51     "Min opening expansion valve (puls),
52     "Postition expansion valve start compressor (puls),
53     "Position expansion valve pressure equalization (puls),
54     "pressure equalization time (sec),
55     "Postition expansion valve 5 - 30 (puls),
56     "Postition expansion valve 5 - 50 (puls),
57     "Postition ex